In [11]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True,
)

examples = [
    {
        "movie" : "Top Gun",
        "answer" : """🛩️👨‍✈️🔥 """,
    },
    {
        "movie" : "The Godfather",
        "answer" : """👨‍👨‍👦🔫🍝"""
    }
]

prompt = ChatPromptTemplate.from_messages(
    [
       ( 'system', "You can explain the movie with only tree emojis"), MessagesPlaceholder(variable_name='history'),
       ("human", "{movie}"),
]
)

# prompt2 = FewShotChatMessagePromptTemplate(
#     example_prompt=prompt,
#     examples=examples,
# )

def load_memory(_):
    return memory.load_memory_variables({})['history']

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(movie):
    result = chain.invoke({'movie': movie})
    memory.save_context(
        {'input': movie},
        {'output': result.content},
    )
    print(result)
    
invoke_chain("interstellar")


content='🚀🌌🌿'


In [12]:
invoke_chain("Dune")

content='🏜️🪐🔮'


In [13]:
invoke_chain("What was the title of the mvie that i asked right before Dune?")

content='Interstellar'
